# Libraries, Settings and Imports

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# account for jupyter autocomplete bug
%config Completer.use_jedi = False

In [3]:
# read in the data
df = pd.read_csv('data/goodreads_library_export.csv')

# Drop Non-Useful Data

In [4]:
# select only the books which have been read
df = df[df['Exclusive Shelf'] == 'read']
# designate the columns which are not useful
unused_cols = [
    'Book Id', 'Author l-f', 'Additional Authors', 'ISBN', 'ISBN13', 'Bookshelves', 'Bookshelves with positions', 'My Review', 'Spoiler', 'Private Notes',
    'Recommended For', 'Recommended By', 'Owned Copies', 'Original Purchase Date', 'Original Purchase Location', 'Condition', 'Condition Description', 'BCID',
    'Year Published', 'Exclusive Shelf', 'Read Count', 'Date Added']
# drop the unused columns
df.drop(unused_cols, axis=1, inplace=True)

# Data Formatting

In [5]:
# ensure all audiobooks have null pages
df.loc[(df['Binding'] == 'Audible Audio') | (df['Binding'] == 'Audiobook'), 'Number of Pages'] = np.nan
# designate any zero ratings as null values
df['My Rating'].replace({0:np.nan}, inplace=True)
# convert the date read column to datetime, ignore nulls
df["Date Read"] = pd.to_datetime(df["Date Read"], format='%Y/%m/%d',errors='ignore')
# convert the original publication year column to datetime
df["Original Publication Year"] = pd.to_datetime(df["Original Publication Year"], format='%Y', errors='ignore')

# Feature Extraction

In [6]:
def book_format_generator(binding):
    """
    This book returns the appropriate book format for its binding type.
    """
    
    if binding in ['Paperback', 'Hardcover']:
        return 'Physical'
    elif binding in ['Kindle Edition']:
        return 'Digital'
    elif binding in ['Audible Audio', 'Audiobook']:
        return 'Audiobook'

In [7]:
def main_title_extractor(title):
    """
    This function extracts the main title from the full Goodreads title.
    """
    
    if '(' and ':' in title:
        return title.split(' (')[0].split(':')[0]
    elif '(' in title:
        return title.split(' (')[0]
    elif ':' in title:
        return title.split(':')[0]
    else:
        return title

In [8]:
def subtitle_extractor(title):
    """
    This title extracts the subtitle, if available, from the full Goodreads title.
    """
    
    if '(' and ':' in title:
        return title.split(' (')[0].split(':')[1]
    elif ':' in title:
        return title.split(':')[1]
    else:
        return np.nan

In [9]:
def series_extractor(title):
    """
    This title extracts the series which the title is a member of, if available, from the full Goodreads title.
    """
    
    if '#' in title:
        return re.search(r'\((.*?)\)', title).group(1).split(', #')[0]
    else:
        return np.nan

In [10]:
# fix binding of wrongly marked books
df.loc[(df['Binding'] == 'ebook') | (df['Binding'] == 'Mass Market Paperback'), 'Binding'] = 'Paperback'
# aggregate book formats together
df['Format'] = df['Binding'].apply(lambda x: book_format_generator(x))

In [11]:
# extract the main title
df['Main Title'] = df['Title'].apply(lambda x: main_title_extractor(x))
# extract the subtitle
df['Subtitle'] = df['Title'].apply(lambda x: subtitle_extractor(x))
# extract the book's series
df['Series'] = df['Title'].apply(lambda x: series_extractor(x))

# Data Cleaning

In [12]:
def publisher_cleaner(publisher):
    """
    This function helps to clean up some duplicate names of publishers.
    """
    if pd.notna(publisher):
        if any(i in publisher for i in ['Penguin', 'Portfolio']):
            return 'Penguin'
        elif 'Puffin' in publisher:
            return 'Puffin'
        elif 'Scholastic' in publisher:
            return 'Scholastic'
        elif 'Random House' in publisher:
            return 'Random House'
        elif any(i in publisher for i in ['Harper', 'Collins', 'Voyager']):
            return 'HarperCollins'
        elif 'Egmont' in publisher:
            return 'Egmont'
        elif 'Bloomsbury' in publisher:
            return 'Bloomsbury'
        elif 'Walker' in publisher:
            return 'Walker'
        elif 'Simon & Schuster' in publisher:
            return 'Simon & Schuster'
        elif 'Vintage' in publisher:
            return 'Vintage'
        else:
            return publisher

In [13]:
# clean up the publisher's names
df['Publisher'] = df['Publisher'].apply(lambda x: publisher_cleaner(x))

In [14]:
# perform any manual edits for data which was spotted to be incorrect
df.loc[df['Main Title'] == 'Evil Star', 'Original Publication Year'] = 2006
df.loc[df['Main Title'] == 'Skeleton Key', 'Series'] = 'Alex Rider'
df.loc[df['Series'] == 'Power of Five', 'Series'] = 'The Power of Five'
df.loc[df['Author'] == 'J.R.R. Tolkien', 'Series'] = 'Middle Earth'
df.loc[df['Subtitle'] == ' Steel and Snow', 'Main Title'] = 'A Storm of Swords 1'
df.loc[df['Subtitle'] == ' Blood and Gold', 'Main Title'] = 'A Storm of Swords 2'
df.loc[df['Subtitle'] == ' Dreams and Dust', 'Main Title'] = 'A Dance with Dragons 1'
df.loc[df['Author'] == 'Michael   Lewis', 'Author'] = 'Michael Lewis'
df.loc[df['Main Title'] == 'Silicon Docks', 'Binding'] = 'Paperback'
df.loc[df['Main Title'] == 'Silicon Docks', 'Format'] = 'Physical'
df.loc[df['Main Title'] == 'Artemis Fowl and the Lost Colony', 'Main Title'] = 'The Lost Colony'
df.loc[df['Main Title'] == 'Artemis Fowl and the Time Paradox', 'Main Title'] = 'The Time Paradox'
df.loc[df['Main Title'] == "Harry Potter and the Philosopher's Stone", 'Series'] = 'Harry Potter'
df.loc[df['Main Title'] == 'Northern Lights', 'Series'] = 'His Dark Materials'
df.loc[df['Main Title'] == "The Amber Spyglass", 'Series'] = 'His Dark Materials'

In [15]:
# export amended file to a new csv
df.to_csv('data/goodreads_library_export_cleaned.csv', index=False)